# Array cable problem

This is how a lot of optimization code starts. A "simple" Jupyter notebook which works. And then we should somehow make this something usable. At least that's my experience. So here we go:

## The problem

In offshore wind, turbines are placed on foundations in the water. They generate electricity, and this electricity has to be brought to shore so we can watch cat videos. The first step is to collect the power from all the turbines in so-called offshore substations (OSS), from where large export cables bring the power to shore. The problem is how to design the network of these "array cables" optimally, i.e. at minimal cost.

## Setup

In [10]:
import json
import pyoptinterface as poi
from pyoptinterface import highs, VariableDomain
from pydantic import BaseModel
import numpy as np

We also need some basic data structures. I could do without but the code would be way too messy. If you have questions, ask:

In [2]:
class Unit(BaseModel):
    name: str
    x: float
    y: float

    def __hash__(self):
        return 100 * hash(self.x) - hash(self.y)

    def is_turbine(self):
        return self.name[:3] == "WTG"

class CableType(BaseModel):
    name: str
    max_mw_on_cable: float
    cost_per_km: float

    def __hash__(self):
        return hash(self.name) + hash(self.max_mw_on_cable) + 5 * hash(self.cost_per_km)

## Getting some data

We are "lucky", we have some test data:

In [3]:
with open("../tests/test_cases/small.json", "r") as file:
    data = json.load(file)

units = [Unit(**u) for u in data["units"]]
cable_types = [CableType(**c) for c in data["cable_types"]]

Now that we have some data, let's build a model!

## Model building
### Objective function

$$
\text{minimize} \quad \sum_{o,d,c} c_cl_{o,d}x_{o,d}^c
$$
where:
- $c_c$ is the cost in \\$/km to use cable type $c$
- $l_{o,d}$ is the length of cable to be built between an origin $o$ and a destination $d$, in km.
- $x_{o,d}^c \in \{0,1\}$ is a binary variable indicating whether a connection between $o$ and $d$ is built using cable type $c$

Note that we have two type of units: we have turbines, and substations. There will never be a connection **from** a substation to a turbine.

In [4]:
model = highs.Model()
x = {(o,d,c): model.add_variable(domain=VariableDomain.Binary, name=f"install_{c} for o in ...

SyntaxError: unterminated f-string literal (detected at line 2) (4267399182.py, line 2)

Yeah, this is horrible to write. So let's not do that. Instead, we add a couple of extra data structures:

In [5]:
class Link(BaseModel):
    origin: Unit
    destination: Unit

    def __hash__(self):
        return hash(self.origin) + 5 * hash(self.destination)

    def get_distance_in_km(self):
        return np.sqrt((self.origin.x - self.destination.x) ** 2 + (self.origin.y - self.destination.y) ** 2) / 1000

class Connection(BaseModel):
    link: Link
    cable_type: CableType

    def __hash__(self):
        return hash(self.link) + 15 * hash(self.cable_type)

    def get_cost(self):
        return self.link.get_distance_in_km() * self.cable_type.cost_per_km

Let's try again:

In [6]:
links = [Link(origin=o, destination=d) for o in units for d in units if o != d and o.is_turbine()]
connections = [Connection(link=link, cable_type=c) for link in links for c in cable_types]

In [7]:
model = highs.Model()
x = {c: model.add_variable(domain=VariableDomain.Binary, name=f"x_{c}") for c in connections}

Running HiGHS 1.9.0 (git hash: 66f735e): Copyright (c) 2024 HiGHS under MIT licence terms


In [11]:
model.set_objective(sum(c.get_cost() * x[c] for c in x), poi.ObjectiveSense.Minimize)

Hessian has dimension 60 but no nonzeros, so is ignored


### Constraints

#### Flow balance

$$
\sum_{o} f_{o,d} + P = \sum_{d'} f_{d,d'} \quad \quad \forall d \in T
$$

where:
- $f_{o,d}$ denotes the flow from unit $o$ to unit $d$
- $T$ is the set of all turbines

In [17]:
f = {f: model.add_variable(lb=0, domain=VariableDomain.Continuous, name=f"f_{f}") for f in links}
turbines = [u for u in units if u.is_turbine()]
turbine_power = 8
for u in turbines:
    model.add_linear_constraint(
        sum(f[l] for l in links if l.destination == u)
        - sum(f[l] for l in links if l.origin == u),
        poi.Eq,
        turbine_power)

#### Flow limit
$$
f_{o,d} \leq \sum_c p_c x_{o,d}^c \quad \quad \forall o \in T, d \in U
$$

where:
- $p_c$ is the maximum allowed power in MW on a given cable
- $U$ is the set of all units

In [22]:
for link in links:
    model.add_linear_constraint(
        f[link]
        - sum(c.cable_type.max_mw_on_cable * x[c] for c in connections if c.link == link),
        poi.Leq,
        0
    )

#### Only one outgoing connection

$$
\sum_{d} y_{o,d} = 1 \quad \quad \forall o \in T
$$

where:
- $y_{o,d} \in \{0,1\}$ indicates whether a link has been built between $o$ and $d$, irrespective of cable type

In [19]:
y = {link: model.add_variable(domain=VariableDomain.Binary, name=f"y_{link}") for link in links}
for u in turbines:
    model.add_linear_constraint(
        sum(y[link] for link in links if link.origin == u),
        poi.Eq,
        1)

#### Connect $x$ and $y$

$$
y_{o,d} = \sum_c x_{o,d}^c \quad \quad \forall o \in T, d \in U
$$

In [20]:
for link in links:
    model.add_linear_constraint(
        sum(x[c] for c in connections if c.link == link) - y[link],
        poi.Eq,
        0)

#### Max number of cables
$$
z_c \leq N
$$

where:
- $z_c \in \{0,1\}$ is a binary variable indicating whether cable type $c$ is built
- $N$ is the maximum number of cables

In [23]:
z = {c: model.add_variable(domain=VariableDomain.Binary, name=f"z_{c}") for c in cable_types}
max_cables = 3
model.add_linear_constraint(sum(z.values()) <= max_cables)

#### Linking $z$ and $x$
$$
x_{o,d}^c \leq z_c \quad \quad \forall c \in C, \forall o \in T, \forall d \in U
$$

where:
- $C$ is the set of all cable types

In [26]:
for cable_type in cable_types:
    for c in [c for c in connections if c.cable_type == cable_type]:
        model.add_linear_constraint(
            x[c] - z[cable_type],
            poi.Leq,
            0
        )

Are we done???? Nope... Now comes the worst constraint of them all: the cables are **not** allowed to cross:

#### No-crossing constraint
$$
y_{o,d} + y_{o',d'} \leq 1 \quad \quad \forall ((o,d),(o',d')) \in Q
$$

where:
- $Q$ is the set of all link combinations which do cross

You'd wanna test that, no???